In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [196]:
owid_df = pd.read_csv('../data/energy-data/owid-energy-data.csv')
gov_response = pd.read_csv('../data/covid-policy-dataset/data/timeseries_indices/OxCGRT_timeseries_GovernmentResponseIndex_v1.csv')

In [180]:
cov_compact = pd.read_csv('../data/covid-policy-dataset/data/OxCGRT_compact_national_v1.csv')

In [4]:
gov_response_countries = gov_response[gov_response['Jurisdiction']=='NAT_TOTAL']

In [5]:
gov_response_rotate = gov_response_countries[gov_response_countries['CountryName'].isin(owid_df['country'])].drop(columns=['CountryCode'
                                                                                                          ,'RegionCode'
                                                                                                          ,'RegionName'
                                                                                                          ,'CityCode'
                                                                                                          ,'CityName'
                                                                                                          ,'Jurisdiction']).transpose().reset_index()

In [6]:
gov_response_rotate.columns = gov_response_rotate.iloc[0]

In [7]:
gov_response_rotate = gov_response_rotate.drop(index=0).rename(columns={'CountryName':'Date'}).reset_index(drop=True)

In [8]:
gov_num = gov_response_rotate.iloc[:,1:].apply(pd.to_numeric)

In [9]:
gov_date = gov_response_rotate.iloc[:,0].apply(pd.to_datetime)

In [10]:
gov_num.insert(0,'Date',gov_date)

In [11]:
gov_num['Year'] = gov_num['Date'].dt.year

In [12]:
gov_mean = gov_num.groupby('Year').mean().reset_index().drop(columns='Date')

In [13]:
gov_melt = gov_mean.melt(id_vars=['Year'],value_vars=gov_mean.columns[1:]).rename(columns={0:'Country'})

In [218]:
owid_trim = owid_df[['country',
                     'year',
                     'coal_consumption',
                     'hydro_consumption',
                     'oil_consumption',
                     'nuclear_consumption',
                     'gas_consumption']]

In [15]:
cov_compact['Year'] = cov_compact['Date'].astype(str).str[:4]

In [16]:
cov_compact['Year'] = cov_compact['Year'].astype(int)

In [17]:
cov_compact.drop(columns=['CountryCode','RegionName','RegionCode','Jurisdiction','Date'])

,CountryName,C1M_School closing,C1M_Flag,C2M_Workplace closing,C2M_Flag,C3M_Cancel public events,C3M_Flag,C4M_Restrictions on gatherings,C4M_Flag,C5M_Close public transport,...,V4_Mandatory Vaccination (summary),ConfirmedCases,ConfirmedDeaths,MajorityVaccinated,PopulationVaccinated,StringencyIndex_Average,GovernmentResponseIndex_Average,ContainmentHealthIndex_Average,EconomicSupportIndex,Year
0,Aruba,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,...,NaN,0.0,0.0,NV,0.00,0.00,0.00,0.00,0.0,2020
1,Aruba,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,...,NaN,0.0,0.0,NV,0.00,0.00,0.00,0.00,0.0,2020
2,Aruba,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,...,NaN,0.0,0.0,NV,0.00,0.00,0.00,0.00,0.0,2020
3,Aruba,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,...,NaN,0.0,0.0,NV,0.00,0.00,0.00,0.00,0.0,2020
4,Aruba,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,...,NaN,0.0,0.0,NV,0.00,0.00,0.00,0.00,0.0,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202755,Zimbabwe,1.0,1.0,2.0,1.0,1.0,1.0,0.0,NaN,0.0,...,1.0,259981.0,5637.0,NV,29.11,29.48,36.44,41.65,0.0,2022
202756,Zimbabwe,1.0,1.0,2.0,1.0,1.0,1.0,0.0,NaN,0.0,...,1.0,259981.0,5637.0,NV,29.11,29.48,36.44,41.65,0.0,2022
202757,Zimbabwe,1.0,1.0,2.0,1.0,1.0,1.0,0.0,NaN,0.0,...,1.0,259981.0,5637.0,NV,29.11,29.48,36.44,41.65,0.0,2022
202758,Zimbabwe,1.0,1.0,2.0,1.0,1.0,1.0,0.0,NaN,0.0,...,1.0,259981.0,5637.0,NV,29.11,29.48,36.44,41.65,0.0,2022


In [18]:
cov_compact_test = cov_compact[cov_compact.columns.drop(list(cov_compact.filter(regex='Flag')))]

In [19]:
cov_compact = cov_compact_test.drop(columns=['CountryCode',
                                             'RegionName',
                                             'RegionCode',
                                             'Jurisdiction',
                                             'Date',
                                             'V2B_Vaccine age eligibility/availability age floor (general population summary)',
                                            'V2C_Vaccine age eligibility/availability age floor (at risk summary)',
                                            'MajorityVaccinated'])

In [20]:
cov_mean = cov_compact.groupby(['CountryName','Year']).mean().reset_index()

In [220]:
owid_trim = owid_trim[(owid_trim['country'].isin(cov_mean['CountryName']))&
                    (owid_trim['year'].between(2018,2020))].rename(columns={'country':'CountryName',
                                                                            'year':'Year'})

In [222]:
owid_trim = owid_trim[owid_trim['gas_consumption'].notnull()]

In [23]:
cov_owid_merge = owid_trim.merge(cov_mean,how='outer',on=('CountryName','Year'))

In [24]:
cov_owid_merge = cov_owid_merge[cov_owid_merge['gas_consumption'].notnull()]

In [25]:
#cov_owid_merge.to_csv('cov_owid_merge.csv',index=False)

In [26]:
#cov_compact[cov_compact['CountryName'].isin(gov_owid_merge['Country'])]

In [198]:
owid_per_capita = (
owid_df[['country',
        'year',
        'coal_cons_per_capita',
        'hydro_energy_per_capita',
        'oil_energy_per_capita',
        'nuclear_energy_per_capita',
        'gas_energy_per_capita']])

In [37]:
owid_sub = owid_trim[(owid_trim['Year'].isin([2019,2021]))&(owid_trim['gas_consumption'].notnull())]

In [38]:
owid_trim_countries = owid_sub[['CountryName','Year']].drop(columns='Year')

In [39]:
owid_sub_2 = owid_sub.groupby('CountryName').diff().dropna().drop(columns='Year').reset_index()

In [40]:
change_calc = owid_trim_countries.reset_index().merge(owid_sub_2,how='inner',on='index').drop(columns='index')

In [41]:
change_calc[change_calc.columns[2:]] = change_calc.iloc[:,2:]/2

In [42]:
change_calc.columns = change_calc.columns.str.replace('_consumption','')

In [43]:
change_calc.insert(1,'flag','calc')

In [44]:
change_calc

,CountryName,flag,coal,hydro,oil,nuclear,gas
0,Algeria,calc,-1.698,-0.1885,-7.8735,0.0000,15.4915
1,Argentina,calc,5.928,-10.5445,14.5295,2.0910,-3.1240
2,Australia,calc,-33.945,2.3795,-34.1420,0.0000,-22.4065
3,Austria,calc,-3.783,-3.1080,-6.4930,0.0000,0.5750
4,Azerbaijan,calc,-0.006,-0.3915,1.0900,0.0000,5.1410
...,...,...,...,...,...,...,...
60,United Kingdom,calc,-4.671,-0.7565,-86.0685,-13.4265,-4.7355
61,United States,calc,-213.566,-50.5150,-209.3490,-47.2235,-72.8750
62,Uzbekistan,calc,4.898,-1.9620,1.6725,0.0000,9.6385
63,Venezuela,calc,-0.323,6.0090,-44.6490,0.0000,12.4145


In [45]:
owid_per_capita = owid_per_capita[owid_per_capita['country'].isin(change_calc['CountryName'])]

In [46]:
cov_mean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 555 entries, 0 to 554
Data columns (total 37 columns):
 #   Column                                              Non-Null Count  Dtype  
---  ------                                              --------------  -----  
 0   CountryName                                         555 non-null    object 
 1   Year                                                555 non-null    int32  
 2   C1M_School closing                                  555 non-null    float64
 3   C2M_Workplace closing                               555 non-null    float64
 4   C3M_Cancel public events                            555 non-null    float64
 5   C4M_Restrictions on gatherings                      555 non-null    float64
 6   C5M_Close public transport                          555 non-null    float64
 7   C6M_Stay at home requirements                       555 non-null    float64
 8   C7M_Restrictions on internal movement               555 non-null    float64
 9  

In [47]:
#cov_mean.to_csv('cov_mean.csv',index=False)

In [48]:
owid_cut = owid_df[['country',
                     'year',
                     'coal_consumption',
                     'hydro_consumption',
                     'oil_consumption',
                     'nuclear_consumption',
                     'gas_consumption']]

In [49]:
#owid_cut[(owid_cut['country'].isin(cov_mean['CountryName']))&(owid_cut['gas_consumption'].notnull())].to_csv('owid_cut.csv',index=False)

In [50]:
owid_cut

,country,year,coal_consumption,hydro_consumption,oil_consumption,nuclear_consumption,gas_consumption
0,ASEAN (Ember),2000,NaN,NaN,NaN,NaN,NaN
1,ASEAN (Ember),2001,NaN,NaN,NaN,NaN,NaN
2,ASEAN (Ember),2002,NaN,NaN,NaN,NaN,NaN
3,ASEAN (Ember),2003,NaN,NaN,NaN,NaN,NaN
4,ASEAN (Ember),2004,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
21807,Zimbabwe,2019,NaN,NaN,NaN,0.0,NaN
21808,Zimbabwe,2020,NaN,NaN,NaN,0.0,NaN
21809,Zimbabwe,2021,NaN,NaN,NaN,0.0,NaN
21810,Zimbabwe,2022,NaN,NaN,NaN,0.0,NaN


In [51]:
fuel_change = owid_df['country'].reset_index().merge(owid_cut[(owid_cut['country'].isin(cov_mean['CountryName']))&
                    (owid_cut['gas_consumption'].notnull())&
                    (owid_cut['year'].isin([2019,2020]))].groupby(['country'])[['hydro_consumption',
                                                                                'coal_consumption',
                                                                                'oil_consumption',
                                                                                'nuclear_consumption',
                                                                                'gas_consumption']].diff()
                                                     .dropna()
                                                     .reset_index(),how='inner',on='index').drop(columns='index')

In [52]:
fuel_change.columns = fuel_change.columns.str.replace('_consumption','')

In [53]:
fuel_change.nsmallest(1,'nuclear')

,country,hydro,coal,oil,nuclear,gas
20,France,14.364,-23.441,-144.009,-117.566,-30.792


In [54]:
owid_cut[(owid_cut['country'].isin(cov_mean['CountryName']))&
                    (owid_cut['gas_consumption'].notnull())&
                    (owid_cut['year'].isin([2019,2020]))&
                    (owid_cut['country']=='China')]

,country,year,coal_consumption,hydro_consumption,oil_consumption,nuclear_consumption,gas_consumption
4501,China,2019,22922.236,3354.375,7891.510,882.026,3083.768
4502,China,2020,23402.553,3471.190,7959.667,922.889,3366.173


In [55]:
fuel_change.merge(change_calc,how='inner',left_on='country',right_on='CountryName',suffixes=('_actual','_calc')).drop(columns='CountryName')

,country,hydro_actual,coal_actual,oil_actual,nuclear_actual,gas_actual,flag,coal_calc,hydro_calc,oil_calc,nuclear_calc,gas_calc
0,Algeria,-0.271,-3.279,-25.026,0.000,-17.046,calc,-1.698,-0.1885,-7.8735,0.0000,15.4915
1,Argentina,-10.338,1.891,-33.637,3.875,-26.277,calc,5.928,-10.5445,14.5295,2.0910,-3.1240
2,Australia,0.916,-18.331,-81.850,0.000,-17.397,calc,-33.945,2.3795,-34.1420,0.0000,-22.4065
3,Austria,2.683,-4.913,-20.172,0.000,-3.732,calc,-3.783,-3.1080,-6.4930,0.0000,0.5750
4,Azerbaijan,-1.316,-0.003,-5.408,0.000,5.776,calc,-0.006,-0.3915,1.0900,0.0000,5.1410
...,...,...,...,...,...,...,...,...,...,...,...,...
60,United Kingdom,2.400,-6.127,-196.564,-15.406,-47.284,calc,-4.671,-0.7565,-86.0685,-13.4265,-4.7355
61,United States,-9.822,-594.279,-1252.363,-59.728,-164.503,calc,-213.566,-50.5150,-209.3490,-47.2235,-72.8750
62,Uzbekistan,-3.905,7.746,4.213,0.000,-10.144,calc,4.898,-1.9620,1.6725,0.0000,9.6385
63,Venezuela,10.949,-0.262,-99.539,0.000,-40.151,calc,-0.323,6.0090,-44.6490,0.0000,12.4145


In [56]:
fuel_change.insert(1,'flag','actual')

In [57]:
#pd.concat([(fuel_change),(change_calc.rename(columns={'CountryName':'country'}))]).sort_values(by='country').to_csv('fuel_change.csv',index=False)

In [58]:
fuel_diff = ((fuel_change.iloc[:,2:])-(change_calc.iloc[:,2:]))

In [59]:
fuel_diff.insert(0,'country',fuel_change['country'])

In [60]:
fuel_diff.insert(1,'flag','diff')

In [61]:
#fuel_diff.to_csv('fuel_diff.csv',index=False)

In [62]:
fuel_change_per_capita = owid_df['country'].reset_index().merge(owid_per_capita[(owid_per_capita['country'].isin(cov_mean['CountryName']))&
                    (owid_per_capita['gas_energy_per_capita'].notnull())&
                    (owid_per_capita['year'].isin([2019,2020]))].groupby(['country'])[['hydro_energy_per_capita',
                                                                                'coal_cons_per_capita',
                                                                                'oil_energy_per_capita',
                                                                                'nuclear_energy_per_capita',
                                                                                'gas_energy_per_capita']].diff()
                                                     .dropna()
                                                     .reset_index(),how='inner',on='index').drop(columns='index')

In [63]:
fuel_change_per_capita.insert(1,'flag','actual')

In [200]:
fuel_change_per_capita_calc = owid_df['country'].reset_index().merge(
    owid_per_capita[(owid_per_capita['year'].isin([2018,2019]))&(owid_per_capita['gas_energy_per_capita'].notnull())]
        .groupby('country')[[
                 'coal_cons_per_capita',
              'hydro_energy_per_capita',
                'oil_energy_per_capita',
            'nuclear_energy_per_capita',
                'gas_energy_per_capita']].diff().dropna().reset_index(),how='inner',on='index').drop(columns='index')

In [204]:
fuel_change_per_capita_calc.insert(1,'flag','expected')

ValueError: cannot insert flag, already exists

In [214]:
(fuel_change_per_capita.iloc[:,2:])/(owid_per_capita[(owid_per_capita['year']==2019)&
                                     (owid_per_capita['gas_energy_per_capita'].notnull())].iloc[:,2:])

,coal_cons_per_capita,gas_energy_per_capita,hydro_energy_per_capita,nuclear_energy_per_capita,oil_energy_per_capita
0,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...
20729,NaN,NaN,NaN,NaN,NaN
20817,NaN,NaN,NaN,NaN,NaN
20984,NaN,NaN,NaN,NaN,NaN
21108,NaN,NaN,NaN,NaN,NaN


In [212]:
owid_per_capita[(owid_per_capita['year']==2019)&(owid_per_capita['gas_energy_per_capita'].notnull())].iloc[:,2:]

,coal_cons_per_capita,hydro_energy_per_capita,oil_energy_per_capita,nuclear_energy_per_capita,gas_energy_per_capita
266,931.140,274.942,1728.642,25.247,1187.398
755,112.473,9.382,5548.513,0.000,10550.106
1131,181.559,1620.299,7192.507,477.227,10405.270
1323,7435.273,1101.867,5373.906,358.301,3243.079
1689,19181.621,1457.515,23811.162,0.000,17374.590
...,...,...,...,...,...
20729,10898.042,2347.308,6680.419,622.988,5002.802
20817,634.166,516.539,1762.390,0.000,13524.053
20984,30.158,5712.858,8099.578,0.000,8828.005
21108,6062.683,1819.676,3482.406,0.000,1027.703


In [67]:
per_capita_diff = ((fuel_change_per_capita.iloc[:,2:])-(fuel_change_per_capita_calc.iloc[:,2:]))

In [68]:
per_capita_diff.insert(0,'country',fuel_change_per_capita['country'])

In [69]:
per_capita_diff.insert(1,'flag','diff')

In [70]:
per_capita_diff

,country,flag,coal_cons_per_capita,gas_energy_per_capita,hydro_energy_per_capita,nuclear_energy_per_capita,oil_energy_per_capita
0,Algeria,diff,-35.226,-923.148,2.447,0.000,-129.836
1,Argentina,diff,-87.982,-390.482,244.790,-3.789,-1350.694
2,Australia,diff,778.944,1217.339,-133.979,0.000,-326.426
3,Austria,diff,-121.423,-531.822,1017.297,0.000,-781.404
4,Azerbaijan,diff,0.297,-404.283,-50.985,0.000,-720.508
...,...,...,...,...,...,...,...
60,United Kingdom,diff,-18.564,-526.096,59.054,176.370,-330.785
61,United States,diff,-1105.885,22.168,277.574,122.565,-2389.552
62,Uzbekistan,diff,97.480,-651.631,6.944,0.000,55.738
63,Venezuela,diff,1.949,-2382.259,-101.732,0.000,-411.998


In [71]:
#pd.concat([fuel_change_per_capita,fuel_change_per_capita_calc,per_capita_diff]).sort_values(by='country').to_csv('change_per_capita.csv',index=False)

In [188]:
cases_deaths= cov_compact[(cov_compact['CountryName'].isin(fuel_change['country']))&(cov_compact['Date'].isin([20201231,20211231,20221231]))][['CountryName',
                                                                                                                                'Date',
                                                                                                                                'ConfirmedCases',
                                                                                                                                'ConfirmedDeaths']]

In [190]:
cases_deaths['Date'] = cases_deaths['Date'].astype(str).str[:4]

In [192]:
cases_deaths.to_csv('cases_deaths.csv',index=False)

In [194]:
cases_deaths[cases_deaths['CountryName']=='United States']

,CountryName,Date,ConfirmedCases,ConfirmedDeaths
192165,United States,2020,20219878.0,350604.0
192530,United States,2021,54907717.0,825468.0
192895,United States,2022,100765333.0,1092764.0


In [186]:
cov_compact[cov_compact['CountryName']=='United States']

,CountryName,CountryCode,RegionName,RegionCode,Jurisdiction,Date,C1M_School closing,C1M_Flag,C2M_Workplace closing,C2M_Flag,...,V3_Vaccine Financial Support (summary),V4_Mandatory Vaccination (summary),ConfirmedCases,ConfirmedDeaths,MajorityVaccinated,PopulationVaccinated,StringencyIndex_Average,GovernmentResponseIndex_Average,ContainmentHealthIndex_Average,EconomicSupportIndex
191800,United States,USA,NaN,NaN,NAT_TOTAL,20200101,0.0,NaN,0.0,NaN,...,0,0.0,0.0,0.0,NV,0.00,0.00,0.00,0.00,0.0
191801,United States,USA,NaN,NaN,NAT_TOTAL,20200102,0.0,NaN,0.0,NaN,...,0,0.0,0.0,0.0,NV,0.00,0.00,0.00,0.00,0.0
191802,United States,USA,NaN,NaN,NAT_TOTAL,20200103,0.0,NaN,0.0,NaN,...,0,0.0,0.0,0.0,NV,0.00,0.00,0.00,0.00,0.0
191803,United States,USA,NaN,NaN,NAT_TOTAL,20200104,0.0,NaN,0.0,NaN,...,0,0.0,0.0,0.0,NV,0.00,0.00,0.00,0.00,0.0
191804,United States,USA,NaN,NaN,NAT_TOTAL,20200105,0.0,NaN,0.0,NaN,...,0,0.0,0.0,0.0,NV,0.00,0.00,0.00,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192891,United States,USA,NaN,NaN,NAT_TOTAL,20221227,1.0,0.0,1.0,0.0,...,5,1.0,100501536.0,1090608.0,V,69.13,28.72,37.25,42.57,0.0
192892,United States,USA,NaN,NaN,NAT_TOTAL,20221228,1.0,0.0,1.0,0.0,...,5,1.0,100614880.0,1091598.0,V,69.13,28.72,37.25,42.57,0.0
192893,United States,USA,NaN,NaN,NAT_TOTAL,20221229,1.0,0.0,1.0,0.0,...,5,1.0,100718983.0,1092522.0,V,69.14,28.72,37.25,42.57,0.0
192894,United States,USA,NaN,NaN,NAT_TOTAL,20221230,1.0,0.0,1.0,0.0,...,5,1.0,100757380.0,1092738.0,V,69.15,28.71,37.25,42.57,0.0


In [224]:
owid_trim

,CountryName,Year,coal_consumption,hydro_consumption,oil_consumption,nuclear_consumption,gas_consumption
754,Algeria,2018,5.815,0.310,227.786,0.000,434.087
755,Algeria,2019,4.803,0.401,236.951,0.000,450.546
756,Algeria,2020,1.524,0.130,211.925,0.000,433.500
1130,Argentina,2018,13.709,85.401,354.626,17.447,487.021
1131,Argentina,2019,8.124,72.501,321.832,21.354,465.589
...,...,...,...,...,...,...,...
20984,Venezuela,2019,0.874,165.511,234.658,0.000,255.762
20985,Venezuela,2020,0.612,176.460,135.119,0.000,215.611
21107,Vietnam,2018,439.514,219.810,324.449,0.000,96.596
21108,Vietnam,2019,580.664,174.283,333.533,0.000,98.430


In [226]:
owid_change_pct = owid_df[['country',
                           'year',
                           'coal_cons_change_pct',
                           'gas_cons_change_pct',
                           'hydro_cons_change_pct',
                           'nuclear_cons_change_pct',
                           'oil_cons_change_pct']]

In [236]:
pct_change_fuel = owid_change_pct[(owid_change_pct['country'].isin(change_calc['CountryName']))&(owid_change_pct['year'].isin([2019,2020]))]

In [256]:
pct_change_fuel_diff = ((pct_change_fuel[pct_change_fuel['year']==2020].iloc[:,2:].reset_index(drop=True))-(pct_change_fuel[pct_change_fuel['year']==2019].iloc[:,2:].reset_index(drop=True)))

In [258]:
pct_change_fuel_diff.insert(0,'country',pct_change_fuel['country'].unique())

In [262]:
pct_change_fuel_diff.to_csv('pct_change_fuel.csv',index=False)